In [1]:
import polars as pl
import plotly.graph_objects as go

## デシル分析とRFM分析による顧客セグメンテーション

### デシル分析

In [6]:
url_data_4_1 = 'https://raw.githubusercontent.com/asakura-data-science/marketing/main/Chapter_4/in/sec4-1data.csv'
df = (
    pl.read_csv(url_data_4_1)
    .with_columns(pl.col('日付').cast(pl.Utf8).str.to_date('%Y%m%d'))
)
df.head()

モニタ,日付,購入数量,単価,金額,大分類,中分類,小分類,細分類,性別,年代,未既婚,大分類名,中分類名,小分類名,細分類名
i64,date,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,str,str
15,2013-07-15,1,118,118,1,14,1403,140397,2,10,3,"""食品""","""飲料・酒類""","""清涼飲料""","""その他清涼飲料"""
15,2013-07-15,1,128,128,1,11,1107,110707,2,10,3,"""食品""","""加工食品""","""冷凍食品""","""冷凍調理"""
15,2013-07-15,1,140,140,2,24,2404,240413,2,10,3,"""日用品""","""家庭用品""","""台所用品""","""キッチンペーパー"""
15,2013-07-15,1,78,78,1,12,1203,120397,2,10,3,"""食品""","""生鮮食品""","""農産""","""その他農産"""
15,2013-07-15,2,83,166,2,26,2622,262201,2,10,3,"""日用品""","""ペット用品""","""猫""","""猫フード"""


In [9]:
# モニタ別金額集計
(
    df.group_by('モニタ').agg(pl.col('金額').sum())
    .sort(by='金額', descending=True)
    .head(10)
)

モニタ,金額
i64,i64
38,582936
57,491733
98,261842
257,242446
37,203218
276,109111
127,97732
575,89702
361,87538


In [10]:
# デシル分析の各ランクの閾値とモニタ別のランクの計算
(
    df.group_by('モニタ').agg(pl.col('金額').sum())
    .quantile()
)

モニタ,金額
i64,i64
265,2444
48,10923
117,3472
15,2880
66,43802
462,156
450,30132
307,11183
436,5345
